In [19]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [20]:
# Path to the data directory
DATA_DIR = Path("../data")

In [21]:
# ---------- Helper Functions ----------
def normalize_name(s: str) -> str:
    """Lowercase, strip, remove punctuation, drop '(feat...)' parts, collapse spaces."""
    if pd.isna(s):
        return ""
    s = s.lower().strip()
    s = re.sub(r"[’'`]", "", s)              # apostrophes and smart quotes
    s = re.sub(r"\s*\(.*?\)\s*", " ", s)     # remove anything in parentheses
    s = re.sub(r"\s+feat\..*", "", s)        # remove 'feat.' trailers
    s = re.sub(r"\s+ft\..*", "", s)          # remove 'ft.' trailers
    s = re.sub(r"[^a-z0-9\s]", " ", s)       # non-alphanum -> space
    s = re.sub(r"\s+", " ", s)               # collapse whitespace
    return s.strip()

def read_csv(path: Path, **kwargs) -> pd.DataFrame:
    df = pd.read_csv(path, **kwargs)
    print(f"Loaded {path.name}: {df.shape[0]:,} rows x {df.shape[1]} cols")
    return df

In [22]:
billboard = read_csv(DATA_DIR / "billboard.csv")
audio = read_csv(DATA_DIR / "audio_features.csv")
spotify = read_csv(DATA_DIR / "spotify_songs.csv")

Loaded billboard.csv: 327,895 rows x 10 cols
Loaded audio_features.csv: 29,503 rows x 22 cols
Loaded spotify_songs.csv: 32,833 rows x 23 cols


### Normalize keys for merging

In [23]:
# Billboard
billboard["song_norm"] = billboard["song"].apply(normalize_name)
billboard["artist_norm"] = billboard["performer"].apply(normalize_name)

# Audio features
audio["song_norm"] = audio["song"].apply(normalize_name)
audio["artist_norm"] = audio["performer"].apply(normalize_name)

# Spotify songs
spotify["song_norm"] = spotify["track_name"].apply(normalize_name)
spotify["artist_norm"] = spotify["track_artist"].apply(normalize_name)

In [24]:
display(billboard.info())
display(audio.info())
display(spotify.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327895 entries, 0 to 327894
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   url                     327895 non-null  object 
 1   week_id                 327895 non-null  object 
 2   week_position           327895 non-null  int64  
 3   song                    327895 non-null  object 
 4   performer               327895 non-null  object 
 5   song_id                 327895 non-null  object 
 6   instance                327895 non-null  int64  
 7   previous_week_position  295941 non-null  float64
 8   peak_position           327895 non-null  int64  
 9   weeks_on_chart          327895 non-null  int64  
 10  song_norm               327895 non-null  object 
 11  artist_norm             327895 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 30.0+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29503 entries, 0 to 29502
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   song_id                    29503 non-null  object 
 1   performer                  29503 non-null  object 
 2   song                       29503 non-null  object 
 3   spotify_genre              27903 non-null  object 
 4   spotify_track_id           24397 non-null  object 
 5   spotify_track_preview_url  14491 non-null  object 
 6   spotify_track_duration_ms  24397 non-null  float64
 7   spotify_track_explicit     24397 non-null  object 
 8   spotify_track_album        24391 non-null  object 
 9   danceability               24334 non-null  float64
 10  energy                     24334 non-null  float64
 11  key                        24334 non-null  float64
 12  loudness                   24334 non-null  float64
 13  mode                       24334 non-null  flo

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  32833 non-null  object 
 1   track_name                32828 non-null  object 
 2   track_artist              32828 non-null  object 
 3   track_popularity          32833 non-null  int64  
 4   track_album_id            32833 non-null  object 
 5   track_album_name          32828 non-null  object 
 6   track_album_release_date  32833 non-null  object 
 7   playlist_name             32833 non-null  object 
 8   playlist_id               32833 non-null  object 
 9   playlist_genre            32833 non-null  object 
 10  playlist_subgenre         32833 non-null  object 
 11  danceability              32833 non-null  float64
 12  energy                    32833 non-null  float64
 13  key                       32833 non-null  int64  
 14  loudne

None

In [25]:
display(billboard.head())
display(audio.head())
display(spotify.head())

,url,week_id,week_position,song,performer,song_id,instance,previous_week_position,peak_position,weeks_on_chart,song_norm,artist_norm
0,http://www.billboard.com/charts/hot-100/1965-0...,7/17/1965,34,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,45.0,34,4,dont just stand there,patty duke
1,http://www.billboard.com/charts/hot-100/1965-0...,7/24/1965,22,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,34.0,22,5,dont just stand there,patty duke
2,http://www.billboard.com/charts/hot-100/1965-0...,7/31/1965,14,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,22.0,14,6,dont just stand there,patty duke
3,http://www.billboard.com/charts/hot-100/1965-0...,8/7/1965,10,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,14.0,10,7,dont just stand there,patty duke
4,http://www.billboard.com/charts/hot-100/1965-0...,8/14/1965,8,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,10.0,8,8,dont just stand there,patty duke


,song_id,performer,song,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity,song_norm,artist_norm
0,-twistin'-White Silver SandsBill Black's Combo,Bill Black's Combo,-twistin'-White Silver Sands,[],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,twistin white silver sands,bill blacks combo
1,¿Dònde Està Santa Claus? (Where Is Santa Claus...,Augie Rios,¿Dònde Està Santa Claus? (Where Is Santa Claus?),['novelty'],NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d nde est santa claus,augie rios
2,......And Roses And RosesAndy Williams,Andy Williams,......And Roses And Roses,"['adult standards', 'brill building pop', 'eas...",3tvqPPpXyIgKrm4PR9HCf0,https://p.scdn.co/mp3-preview/cef4883cfd1e0e53...,166106.0,False,The Essential Andy Williams,0.154,...,0.0315,0.91100,0.000267,0.112,0.150,83.969,4.0,38.0,and roses and roses,andy williams
3,...And Then There Were DrumsSandy Nelson,Sandy Nelson,...And Then There Were Drums,"['rock-and-roll', 'space age pop', 'surf music']",1fHHq3qHU8wpRKHzhojZ4a,NaN,172066.0,False,Compelling Percussion,0.588,...,0.0361,0.00256,0.745000,0.145,0.801,121.962,4.0,11.0,and then there were drums,sandy nelson
4,...Baby One More TimeBritney Spears,Britney Spears,...Baby One More Time,"['dance pop', 'pop', 'post-teen pop']",3MjUtNVVq3C8Fn0MP3zhXa,https://p.scdn.co/mp3-preview/da2134a161f1cb34...,211066.0,False,...Baby One More Time (Digital Deluxe Version),0.759,...,0.0307,0.20200,0.000131,0.443,0.907,92.960,4.0,77.0,baby one more time,britney spears


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,song_norm,artist_norm
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754,i dont care loud luxury remix,ed sheeran
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600,memories dillon francis remix,maroon 5
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616,all the time don diablo remix,zara larsson
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093,call you mine keanu silva remix,the chainsmokers
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052,someone you loved future humans remix,lewis capaldi


In [47]:
# Billboard unique & hit label skeleton (we'll finalize in Section 2)
bb_unique = (billboard
             .groupby(["song_norm", "artist_norm"], as_index=False)
             .agg(min_rank=("week_position", "min"))
            )
bb_unique["hit"] = (bb_unique["min_rank"] <= 10).astype(int)

print("\nBillboard unique tracks:", len(bb_unique))
print("Billboard Top-10 hits:", int(bb_unique['hit'].sum()))


Billboard unique tracks: 29363
Billboard Top-10 hits: 4980


### Check overlap between datasets to inform merging strategy

In [30]:
# ID-based overlap between spotify_songs.track_id and audio_features.spotify_track_id
spotify_id_set = set(spotify["track_id"].dropna().unique())
audio_id_set = set(audio["spotify_track_id"].dropna().unique())
print(f"\nID overlap (spotify.track_id ↔ audio.spotify_track_id): {len(spotify_id_set & audio_id_set):,}")

# Name-based overlap between audio_features and spotify_songs
audio_spotify_name_overlap = audio.merge(
    spotify[["song_norm", "artist_norm"]].drop_duplicates(),
    on=["song_norm", "artist_norm"], how="inner"
)
print(f"Name overlap (Audio ↔ Spotify): {len(audio_spotify_name_overlap):,}")


ID overlap (spotify.track_id ↔ audio.spotify_track_id): 2,698
Name overlap (Audio ↔ Spotify): 2,647


In [31]:
# Name-based overlap between datasets
audio_name_overlap = bb_unique.merge(
    audio[["song_norm", "artist_norm"]].drop_duplicates(),
    on=["song_norm", "artist_norm"], how="inner"
)
spotify_name_overlap = bb_unique.merge(
    spotify[["song_norm", "artist_norm"]].drop_duplicates(),
    on=["song_norm", "artist_norm"], how="inner"
)

print(f"Name overlap (Billboard ↔ Audio): {len(audio_name_overlap):,}")
print(f"Name overlap (Billboard ↔ Spotify): {len(spotify_name_overlap):,}")

Name overlap (Billboard ↔ Audio): 29,350
Name overlap (Billboard ↔ Spotify): 2,603


#### Trim datasets to relevant columns for predictive modeling

In [29]:
# Trim billboard.csv to relevant columns
# Dropped columns (with rationale):
# - url                           → just a link to Billboard site, irrelevant
# - song_id                       → concatenated string (song+artist), redundant
# - instance                      → technical weekly index, not useful for modeling
# - previous_week_position        → redundant once we use peak_position and weeks_on_chart

billboard = billboard[[
    "song", "performer", "week_id",
    "week_position", "peak_position", "weeks_on_chart",
    "song_norm", "artist_norm"
]]


# Trim audio_features.csv to relevant columns
# Dropped columns (with rationale):
# - song_id, performer, song        → redundant raw text, replaced by normalized keys
# - spotify_track_album             → album name, not predictive
# - spotify_track_preview_url       → preview link, irrelevant
# - spotify_track_explicit          → explicit flag, weak signal for Billboard success
# - time_signature, key, mode       → music theory attributes, low interpretability/business value

audio = audio[[
    "spotify_track_id",
    "danceability", "energy", "loudness", "valence", "tempo",
    "acousticness", "speechiness", "instrumentalness",
    "spotify_track_duration_ms", "spotify_genre",
    "spotify_track_popularity",
    "song_norm", "artist_norm"
]]


# Trim spotify_songs.csv to relevant columns
# Dropped columns (with rationale):
# - track_album_id, track_album_name, track_album_release_date → album-level info, not predictive for hit success
# - playlist_id, playlist_name       → playlist identity too granular; no playlist popularity metadata available
# - key, mode, speechiness, acousticness, instrumentalness, liveness 
#       → secondary audio features, increase dimensionality with low business interpretability

spotify = spotify[[
    "track_id", "track_name", "track_artist",
    "track_popularity", "playlist_genre", "playlist_subgenre",
    "danceability", "energy", "loudness", "valence", "tempo", "duration_ms",
    "song_norm", "artist_norm"
]]

#### Billboard label creation + clean base dataset

In [32]:
# Fail-safe: expected columns
req_cols = {"song_norm","artist_norm","week_id","week_position","peak_position","weeks_on_chart","song","performer"}
missing = req_cols - set(billboard.columns)
assert not missing, f"Billboard missing columns: {missing}"

In [38]:
# Parse dates and coerce numerics
billboard["week_id_dt"] = pd.to_datetime(billboard["week_id"], errors="coerce")
billboard["week_position"] = pd.to_numeric(billboard["week_position"], errors="coerce")
billboard["peak_position"]  = pd.to_numeric(billboard["peak_position"], errors="coerce")
billboard["weeks_on_chart"] = pd.to_numeric(billboard["weeks_on_chart"], errors="coerce")

In [43]:
# Drop rows without normalized keys or rank
billboard_clean = billboard.dropna(subset=["song_norm","artist_norm","week_position"]).copy()
print(f"Rows dropped from billboard: {billboard.shape[0] - billboard_clean.shape[0]}")

Rows dropped from billboard: 0


In [46]:
# 2) Aggregate to one row per (song_norm, artist_norm)
# - min_rank: best weekly chart position
# - peak_final: sanity check against Billboard's peak_position column (min over all weeks)
# - first_week / debut_rank: entry timing & rank
# - weeks_on_chart_total: longevity
agg_min = (billboard_clean
           .groupby(["song_norm","artist_norm"], as_index=False)
           .agg(
               min_rank=("week_position","min"),
               peak_final=("peak_position","min"),
               first_week=("week_id_dt","min"),
               weeks_on_chart_total=("weeks_on_chart","max"),
               # Keep a representative display name for readability
               song_display=("song","first"),
               artist_display=("performer","first"),
           ))

# Debut rank (week_position at first_week)

# Find the first record per song/artist to get debut rank
first_rows = (
    billboard_clean.sort_values(["song_norm","artist_norm","week_id_dt"])
    .groupby(["song_norm","artist_norm"], as_index=False)
    .first()[["song_norm","artist_norm","week_position"]]
    .rename(columns={"week_position":"debut_rank"})
)
bb_base = agg_min.merge(first_rows, on=["song_norm","artist_norm"], how="left")

In [48]:
# Create binary label: Top-10 hit
bb_base["hit"] = (bb_base["min_rank"] <= 10).astype(int)

In [49]:
# Sanity checks
n_total   = len(bb_base)
n_hits    = int(bb_base["hit"].sum())
n_missing = int(bb_base[["min_rank","debut_rank","weeks_on_chart_total"]].isna().any(axis=1).sum())

print(f"Billboard base created: {n_total:,} unique (song, artist) pairs")
print(f"Top-10 hits: {n_hits:,}  ({n_hits/n_total:.1%})")
print(f"Rows with any missing key aggregates (min_rank/debut_rank/weeks_on_chart_total): {n_missing:,}")

Billboard base created: 29,363 unique (song, artist) pairs
Top-10 hits: 4,980  (17.0%)
Rows with any missing key aggregates (min_rank/debut_rank/weeks_on_chart_total): 0
